In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Movie example using write_beam

Here we insert write_beam elements into an existing lattice, run, save the beams to an h5 file, and plot using openPMD-beamphysics tools

In [2]:
from impact import Impact
from impact.parsers import parse_impact_particles
from impact.writers import write_impact_particles_h5

import numpy as np
import os

In [3]:
ipath = '../templates/lcls_injector/'
ifile = '../templates/lcls_injector/ImpactT.in'
I = Impact(ifile, verbose=False)

/Users/chrisonian/Code/GitHub/lume-impact/templates/lcls_injector


In [4]:
# Change some things
I.input['header']['Np'] = 3004
I.input['header']['Nx'] = 16
I.input['header']['Ny'] = 16
I.input['header']['Nz'] = 16
I.input['header']['Dt'] = 5e-13
# Change stop location
I.ele['stop_1']['s'] = 0.1


In [5]:
# Make new elements
def new_write_beam(s, i, sample_frequency=1):
    if i in [40, 50]:
        print('Warning: will overwrite standard output files')
    ele = {'type':'write_beam', 'sample_frequency':sample_frequency}
    ele['filename'] = 'fort.'+str(i)
    ele['name'] = 'write_beam_'+str(i)
    ele['s'] = s
    return ele
def new_write_beams(slist, istart, sample_frequency=1):
    eles = []
    if len(slist) > 100:
        print('Warning: the maximum number of phase- space files which can be output is 100.')
    i = istart
    for s in slist:
        ele = new_write_beam(s, i, sample_frequency)
        eles.append(ele)
        i += 1
    return eles
slist = np.linspace(0.01, 0.1, 99)
neweles = new_write_beams(slist, 100)

In [6]:
# Prepend these to the lattice
I.input['lattice'] = neweles + I.input['lattice']

In [7]:
I.timeout = 1000
I.run()

# Write to h5

In [8]:
import h5py

In [9]:
H5FILE = I.path+'/test.h5'

with h5py.File(H5FILE, 'w') as h5:
    for e in neweles:
        fname = I.path+'/'+e['filename']
        if os.path.exists(fname):
            print('writing', fname)
            particle_data = parse_impact_particles(fname)
            write_impact_particles_h5(h5, particle_data, name=e['name'])

writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.100
writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.101
writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.102
writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.103
writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.104
writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.105
writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.106
writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.107
writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.108
writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.109
writing /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_8krkvxbr/fort.110
writing /var/folders/wj/lfgr0199

# Plot

In [10]:
from bokeh.plotting import  show, figure, output_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource
output_notebook(verbose=False, hide_banner=True)
import os

from opmd_beamphysics import bunch_plotting, bunch_tools, bunch_stats

In [11]:
h5 = h5py.File(H5FILE, 'r')

In [12]:
bunch1 = h5['write_beam_100']
show(
    column(
        
    bunch_plotting.plot_bunch_h5(bunch1, 'z', 'x', bins = 50, liveOnly=False),
    bunch_plotting.plot_histogram_h5(bunch1, 'x', bins=100,liveOnly=False),
    bunch_plotting.plot_bunch_h5(bunch1, 'z', 'pz', bins = 50, liveOnly=False),
   # bunch_plotting.plot_bunch_current_profile(bunch1, bins=100, liveOnly=False)
    
    
    ))

# Interactive 

In [13]:
from bokeh.models.widgets import Slider
from bokeh import palettes, colors
pal = palettes.Viridis[256]
white=colors.named.white
pal[0] = white # replace 0 with white

In [14]:
from opmd_beamphysics.bunch_tools import nice_phase_space_label
from opmd_beamphysics.bunch_plotting import bin_bunch_datasource_h5

In [19]:
# interactive
def myapp2(doc):
    
    bunches = list(h5)
    
    doc.h5 = bunch1
    doc.component1 = 'x'
    doc.component2 = 'px' 
    doc.xlabel = nice_phase_space_label[doc.component1]
    doc.ylabel =nice_phase_space_label[doc.component2]      

    doc.bins = 100
    
    #doc.range = FULLRANGE
    ds  = bin_bunch_datasource_h5(doc.h5, doc.component1, doc.component2,bins=doc.bins, liveOnly = False) 

    def refresh():
        ds.data  = bin_bunch_datasource_h5(doc.h5, doc.component1, doc.component2,bins=doc.bins, liveOnly = False).data 
    
    # Default plot
  
    plot = figure(title='',  
                  x_axis_label = doc.xlabel, y_axis_label = doc.ylabel,
           plot_width=500, plot_height=500)    
    plot.image(image='image', x='xmin', y='ymin', dw='dw', dh='dh', source=ds, palette=pal)
    
    def slider_handler(attr, old, new):
        doc.h5 = h5[bunches[new]]
        refresh()

    slider = Slider(start=0, end=len(bunches)-1, value=0, step=1, title='x')             
    slider.on_change('value', slider_handler)

    # Add plot to end
    doc.add_root(column(slider, plot))
show(myapp2)# , notebook_url=remote_jupyter_proxy_url)